In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('cali_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_hospitalized_patients', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_10_min', 'current_intensive_care_patients', 'relative_humidity', 'vaccination_rate_complete', 'new_deaths_50_mean', 'new_deaths_100_std', 'Completeness_pct', 'new_deaths_10_mean', 'new_deaths_100_mean', 'Booster_Doses_18Plus', 'average_temperature_celsius', 'new_deaths_50_min', 'new_cases_50_max', 'new_cases_100_mean', 'new_cases_50_mean', 'vaccination_rate_1dose', 'new_deaths_10_std', 'new_cases_50_std', 'new_deaths_50_std', 'Administered_Dose1_Recip_12Plus', 'dew_point', 'maximum_temperature_celsius', 'minimum_temperature_celsius', 'new_deaths_100_max', 'Administered_Dose1_Recip_65Plus']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_mean', 'new_hospitalized_patients', 'relative_humidity', 'average_temperature_celsius', 'new_cases_10_max', 'new_cases_10_min', 'minimum_temperature_celsius', 'new_deaths_10_mean', 'new_deaths_10_std', 'new_cases_50_std', 'current_intensive_care_patients', 'maximum_temperature_celsius', 'dew_point', 'new_cases_50_mean', 'new_deaths_50_mean', 'new_cases_100_mean', 'Booster_Doses', 'Series_Complete_5Plus', 'new_deaths_50_std', 'new_deaths_100_std', 'Booster_Doses_50Plus', 'Administered_Dose1_Recip_5Plus', 'new_deaths_100_mean', 'new_deaths_10_min', 'new_deaths_10_max', 'new_cases_100_std', 'Booster_Doses_18Plus', 'new_cases_50_min']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'current_intensive_care_patients', 'new_deaths_10_mean', 'new_hospitalized_patients', 'new_deaths_100_mean', 'relative_humidity', 'dew_point', 'new_cases_10_min', 'new_cases_50_mean', 'new_cases_50_std', 'rainfall_mm', 'new_deaths_10_std', 'Administered_Dose1_Recip_12Plus', 'new_deaths_50_mean', 'average_temperature_celsius', 'new_cases_100_mean', 'minimum_temperature_celsius', 'stay_at_home_requirements', 'vaccination_rate_1dose', 'Series_Complete_5Plus', 'new_deaths_100_std', 'new_cases_100_max', 'Administered_Dose1_Recip_65Plus', 'new_deaths_50_std', 'new_cases_100_std', 'new_deaths_50_max', 'contact_tracing']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_cases_10_mean', 'new_cases_10_std', 'new_cases_10_max', 'new_deaths_100_mean', 'new_cases_50_std', 'new_hospitalized_patients', 'PEOPLE_DEATH_NEW_COUNT', 'dew_point', 'new_deaths_10_mean', 'new_cases_10_min', 'average_temperature_celsius', 'new_deaths_50_mean', 'minimum_temperature_celsius', 'relative_humidity', 'current_intensive_care_patients', 'new_deaths_10_std', 'new_deaths_50_std', 'new_cases_50_min', 'rainfall_mm', 'new_cases_50_mean', 'new_deaths_100_std', 'new_cases_100_mean', 'new_deaths_50_max', 'maximum_temperature_celsius', 'new_cases_100_std', 'Series_Complete_5Plus', 'new_cases_100_max', 'Administered_Dose1_Recip_12Plus', 'Series_Complete_65Plus', 'new_deaths_10_max']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_hospitalized_patients', 'new_cases_10_std', 'new_deaths_100_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_50_std', 'current_intensive_care_patients', 'new_deaths_50_mean', 'new_cases_10_min', 'dew_point', 'new_deaths_10_mean', 'relative_humidity', 'new_deaths_10_std', 'new_deaths_50_std', 'new_deaths_100_std', 'new_cases_100_mean', 'minimum_temperature_celsius', 'new_cases_50_mean', 'average_temperature_celsius', 'rainfall_mm', 'new_cases_100_std', 'maximum_temperature_celsius', 'Booster_Doses', 'new_deaths_10_min', 'new_deaths_10_max', 'Administered_Dose1_Recip_18Plus', 'booster_rate', 'Series_Complete_5Plus', 'Series_Complete_12Plus']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
--------------------------------------------------------------------
Basis Function                                Pruned  Coefficient   
--------------------------------------------------------------------
(Intercept)                                   No      1.29818e+06   
h(new_cases_10_mean-28728)                    No      0.851745      
h(28728-new_cases_10_mean)                    No      -1.39325      
h(new_cases_50_mean-33036.2)                  No      -1.5207       
h(33036.2-new_cases_50_mean)                  No      0.694132      
h(Booster_Doses-7.30053E+06)                  Yes     None          
h(7.30053E+06-Booster_Doses)                  Yes     None          
h(PEOPLE_DEATH_NEW_COUNT-292)                 Yes     None          
h(292-PEOPLE_DEATH_NEW_COUNT)                 No      -24.9042      
h(new_deaths_10_mean-133)                     No      -32.4465      
h(133-new_deaths_10_mean)                     No      24.663        
h(new_cases_10_min-61)

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'new_hospitalized_patients', 'new_deaths_100_std', 'new_cases_10_mean', 'new_cases_10_std', 'new_cases_10_min', 'dew_point', 'new_deaths_50_mean', 'new_cases_50_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_deaths_100_mean', 'new_deaths_10_std', 'average_temperature_celsius', 'new_cases_50_mean', 'new_deaths_10_mean', 'relative_humidity', 'current_intensive_care_patients', 'new_cases_10_max', 'new_deaths_50_std', 'new_cases_100_mean', 'minimum_temperature_celsius'}


## XGBoost Feature Selection, includes lag and rolling

In [ ]:
import os
os._exit(00)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
covid_df = pd.read_csv('cali_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [3]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['new_cases_10_mean', 'new_cases_7_lag', 'new_cases_2_lag', 'new_cases_10_max', 'new_cases_3_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_1 lag', 'new_hospitalized_patients', 'new_cases_10_std', 'new_cases_4_lag', 'new_cases_10_min', 'new_deaths_50_lag', 'new_cases_150_lag', 'new_deaths_7_lag', 'new_deaths_50_max', 'new_deaths_100_std', 'new_deaths_1_lag', 'current_intensive_care_patients', 'new_deaths_50_mean', 'Administered_Dose1_Recip_18Plus', 'Booster_Doses', 'maximum_temperature_celsius', 'new_cases_50_lag', 'new_deaths_100_mean', 'new_cases_50_std', 'new_cases_30_lag', 'new_cases_100_max', 'Series_Complete_Yes', 'new_cases_100_std', 'new_deaths_4_lag', 'Series_Complete_18Plus', 'Administered_Dose1_Recip_12Plus', 'booster_rate', 'new_deaths_3_lag', 'new_deaths_50_std', 'dew_point', 'new_cases_100_min', 'relative_humidity', 'average_temperature_celsius', 'Booster_Doses_65Plus']


In [4]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_10_std', 'new_cases_7_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_2_lag', 'new_cases_4_lag', 'new_cases_10_mean', 'new_cases_1 lag', 'new_cases_3_lag', 'new_deaths_3_lag', 'new_deaths_2_lag', 'new_cases_10_max', 'new_deaths_4_lag', 'new_cases_10_min', 'new_deaths_1_lag', 'new_cases_150_lag', 'new_cases_50_std', 'new_deaths_30_lag', 'new_deaths_7_lag', 'new_deaths_50_lag', 'current_intensive_care_patients', 'new_cases_30_lag', 'minimum_temperature_celsius', 'new_hospitalized_patients', 'dew_point', 'new_cases_350_lag', 'new_cases_50_lag', 'relative_humidity', 'average_temperature_celsius', 'new_deaths_10_min', 'new_deaths_10_mean']


In [5]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_cases_7_lag', 'new_cases_100_mean', 'new_deaths_100_max', 'new_deaths_100_mean', 'new_hospitalized_patients', 'current_intensive_care_patients', 'new_cases_10_std', 'new_cases_2_lag', 'new_deaths_10_mean', 'new_cases_50_std', 'new_cases_1 lag', 'new_cases_10_min', 'new_deaths_50_lag', 'new_cases_3_lag', 'new_cases_150_lag', 'rainfall_mm', 'new_cases_30_lag', 'new_deaths_30_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_deaths_100_std', 'new_cases_50_mean', 'new_deaths_2_lag', 'Administered_Dose1_Recip_5Plus', 'new_cases_4_lag', 'new_deaths_50_mean', 'maximum_temperature_celsius', 'new_deaths_10_std', 'Booster_Doses']


In [6]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
---------------------------------------------------------------------------
Basis Function                                       Pruned  Coefficient   
---------------------------------------------------------------------------
(Intercept)                                          No      6.27038e+06   
h(new_cases_10_mean-28728)                           No      2.92069       
h(28728-new_cases_10_mean)                           No      -2.99535      
h(new_cases_50_mean-33036.2)                         No      4.55891       
h(33036.2-new_cases_50_mean)                         No      1.0835        
h(Booster_Doses-7.30053E+06)                         No      0.485186      
h(7.30053E+06-Booster_Doses)                         No      -0.485584     
present(new_cases_7_lag)                             Yes     None          
missing(new_cases_7_lag)                             No      -6985.18      
h(new_cases_7_lag-19743)*present(new_cases_7_lag)    No      -0.103843     


In [7]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [8]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [9]:
full = common_member(a, b, c)
full

{'new_cases_150_lag', 'new_cases_4_lag', 'new_cases_10_min', 'new_cases_50_std', 'new_cases_10_std', 'new_cases_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_3_lag', 'new_cases_7_lag', 'new_cases_10_max', 'new_deaths_50_lag', 'current_intensive_care_patients', 'new_cases_1 lag', 'new_cases_2_lag', 'new_cases_30_lag', 'new_hospitalized_patients'}
